In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import numpy as np
import os

BATCH_SIZE = 32
IMG_SIZE = 224

In [2]:
data_dir = os.path.join(os.getcwd(), '..', 'data', 'img', 'vit')

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='binary',  # since you have a binary classification problem
    class_names=['real', 'fake'],  # specify the folders to include
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
)

Found 8638 files belonging to 2 classes.
Using 6911 files for training.


In [4]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='binary',
    class_names=['real', 'fake'],
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
)

Found 8638 files belonging to 2 classes.
Using 1727 files for validation.


In [5]:
def preprocess_input(image, label):
    image = tf.keras.layers.Rescaling(1./127.5, offset=-1)(image)
    return image, label

In [6]:
train_dataset = train_dataset.map(preprocess_input)
validation_dataset = validation_dataset.map(preprocess_input)

In [7]:
# convert to numpy
train_dataset = train_dataset.as_numpy_iterator()
validation_dataset = validation_dataset.as_numpy_iterator()

In [8]:
# get the first batch
images, labels = next(train_dataset)

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 6 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: 7 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: 5 extraneous bytes before marker 0x

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 6 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of d

In [ ]:
# # Configure the dataset for performance
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
model_url = "/kaggle/input/vision-transformer/tensorflow2/vit-b8-fe/1"
base_model = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b8-fe/versions/1", trainable=False)

In [11]:
base_model.tra

<bound method KerasLayer._setup_layer of <tensorflow_hub.keras_layer.KerasLayer object at 0x179058950>>

In [ ]:
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    base_model,
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=['accuracy']
)


In [ ]:
model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset
)

In [ ]:
base_model = tf.keras.applications.ResNet152V2(
    weights='imagenet',
    input_shape=(img_height, img_width, 3),
    include_top=False)

base_model.trainable = False

inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(loss='binary_crossentropy'
              , optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5), metrics='binary_accuracy')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])